# Pretrained Networks

![CNN-Architecture-over-a-timeline](https://www.aismartz.com//blog/wp-content/uploads/2019/10/CNN-Architecture-over-a-timeline.jpg)

https://www.aismartz.com/blog/cnn-architectures/

+ WHAT?
    + it is a network which was trained on a large dataset on a large-scale-image classification Task. One can usit as it is for image classification or for transfer learning so we can custumise the model for a new task
    
+ BENEFITS
    + we don't have to train the model
    + very easy to incorporate 
    + fast simulation
    + we can achieve very good performaces
    
+ EXAMPLE
 + U net: recognize feature on medical images
 + MobileNet
 + VGG16/19
 + ResNet
 + InceptionV3
 
 [1000 different classes](https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a) that were used from the [Imagenet](https://www.image-net.org/update-mar-11-2021.php) dataset for training some of the pretrained models

In [ ]:
import numpy as np
from tensorflow.keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing import image # Keras own inbuild image class
import matplotlib.pyplot as plt
import os
from tensorflow.keras.models import Model
from tensorflow import keras

#### Load image from internet that belong under any of the 1000 classes

In [ ]:
#convert image to array, can also specify datatype


In [ ]:
#plot image 


#### Load model

In [ ]:
img.shape

In [ ]:
#show model summary

### Preprocess

In [ ]:
#https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet50/preprocess_input
img = 

In [ ]:
plt.imshow(img)

In [ ]:
#check shape required by model

##### Expand dimensions

#### Predict

In [ ]:
pred =

In [ ]:
#shape of pred

In [ ]:
# decode labels


## Let's have a look at the output of an intermediate convolution layer

In [ ]:
#view model layers

In [ ]:
# choose your intermediate convolution layer to visualize the output at that layer
layer_name = 

In [ ]:
# Initial input:
model.input

In [ ]:
# Input to our chosen layer:
model.get_layer(layer_name).input

In [ ]:
layer_output = model.get_layer(layer_name).output
layer_output

In [ ]:
intermediate_model = keras.models.Model(inputs=model.input, outputs=layer_output)

In [ ]:
intermediate_model.summary()

In [ ]:
# Feature map:
feature_map=intermediate_model.predict(img)
feature_map.shape

In [ ]:
plt.imshow(feature_map[0,:,:,63])

# Transfer Learning

## Transfer Learning for Neural Networks

> Transfer learning consists of taking features learned on one problem, and leveraging them on a new, similar problem. For instance, features from a model that has learned to identify racoons may be useful to kick-start a model meant to identify tanukis (japanese racoons).

__The benefits of transfer learning are:__
* you can reuse pre-trained networks
* it saves lots of training time
* it allows you to train with very small training datasets

__Procedure__
1. Take the weights and architecture of a [pre-trained network](https://keras.io/api/applications/)
2. Load the "convolutional base" of the model (everything except the final dense layers)
3. Freeze all the layers of the base (weights become fixed)
4. Add a fully connected dense layer on top
5. **Add a task specific dense output layer**
6. Compile and fit the model to your data

## Load images into `keras`
Keras has its own in build Objects and Methods to get image data in efficiently
See: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image

- `class ImageDataGenerator`: Generate batches of tensor image data with **optional** real-time data augmentation

In [ ]:
# folder names containing images of the things you want to classify

# plug in the path to your data folder


In [ ]:
# define an image data generator
data_gen = image.ImageDataGenerator(
    # define the preprocessing function that should be applied to all images
    preprocessing_function=preprocess_input,
     #rotation_range=45,
     #width_shift_range=0.2,
     #height_shift_range=0.2,
     #horizontal_flip=True, 
     #vertical_flip=True,
    #zoom_range=0.2,
    # shear_range=0.2    
)

In [ ]:
# a generator that returns batches of X and y arrays
train_data_gen = data_gen.flow_from_directory(
        directory=base_path,
        class_mode="categorical",
        classes=classes,
        batch_size=30,
        target_size=(224, 224)
)

In [ ]:
# load in all images at once
xtrain, ytrain = next(train_data_gen)
xtrain.shape, ytrain.shape

In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(6, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(xtrain)
print(ytrain)

In [ ]:
classes

In [ ]:
ytrain

## Create CNN Model

### 1. Select the convolutional base 

In [ ]:
base_model = ResNet50()

### 2. Freeze the weights

In [ ]:
base_model.summary()

In [ ]:
# freeze it!
base_model.trainable = False

### 3. Add your own dense layers on top

In [ ]:
len(classes)

In [ ]:
model = keras.Sequential()
model.add(base_model)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(len(classes), activation='softmax')) #!!! Final layer with a length of 2, and softmax activation 
# have a look at the trainable and non-trainable params statistic
model.summary()

### 4. Compile and train!

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])

# observe the validation loss and stop when it does not improve after 3 iterations
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

model.fit(xtrain, ytrain, 
          epochs=50, 
          verbose=2,
          callbacks=[callback],
          # use 30% of the data for validation
          validation_split=0.3)

### (5. Use it to predict)

In [ ]:
img = image.load_img('object.png',target_size=(224,224))

In [ ]:
plt.imshow(img)

In [ ]:
img.size

In [ ]:
a = image.img_to_array(img,dtype='uint8')

In [ ]:
a = preprocess_input(a)

In [ ]:
a = np.expand_dims(a, axis = 0)

In [ ]:
a.shape

In [ ]:
model.predict(a)

In [ ]:
model.predict(a)[0].round(decimals = 3)

In [ ]:
classes

In [ ]:
plt.bar(x = classes, height = model.predict(a)[0])

### (6. Save your model for later)

In [ ]:
model.save('models/wallet_phone.h5')

---
### Advanced Optional Step: Fine Tuning

This is done after the initial training! Adapt a few of the base layers to the specific learning task by retraining the model. This can improve accuracy, especially if the original learning task of the pre-trained model differs a lot from the actual task.

1. Unfreeze some (or all) of the layers in the convolutional base (starting with the base output layer)
2. Recompile your model and choose a very low learning rate (`1e-5`)
2. Continue training the model but stop early to avoid overfitting

#### (Advanced: Data augmentation)

> https://keras.io/guides/transfer_learning/

> https://www.tensorflow.org/tutorials/images/data_augmentation

Applies random distortions and transformations to the images (only on your training data!). You need to store your training and validation data at separate locations and use a second `ImageDataGenerator` for your validation data. 

---
---

## Our project: How to continue!

1. Have image data in the categories we want to classify: check
2. Have a model to use:
    - One or several pre-trained models
    - Train your own with transfer learning <-- will probably have the best performance
    - Take the challenge and train your own from scratch
4. Test on your data, evaluate, reflect

5. Save the model(s)

6. Load the trained model into `predict.py` (the modified `capture.py` with the `predict_frame(frame)` function) 


- If you don't have it yet, write a fuction `predict_frame(frame)` that uses the trained model to predict the object in the current frame. It should return a dictionary of class probabilities and names.
    - make sure that the input image to the model is of size (224, 224)
- Modify the script such that it makes a prediction once you press the `p` key
- Write the prediction as a log message to the terminal

### Advanced

- Display the result of the prediction on the current webcam frame
- Make an automatic prediction every second (Hint: the `while` loop has a speed of approx. 30 frames per second)

# Loading the entire images and labels into arrays

In [ ]:
# Let's explore the data folder
base_path = 'data/'

# Let's define the classes
classes = os.listdir(base_path)

In [ ]:
 for class_ in classes:
        print(class_)

In [ ]:
def load_image(base_path):
    """it loads all the image into X and the classes in y """
    X_list = []
    y_list = []
    classes = os.listdir(base_path)
    for class_ in classes:
        
        files = os.listdir(base_path+class_)
        for file in files:
            pic = image.load_img(path=base_path+class_+'/'+f'{file}',target_size=(224,224))
            numpy_image = np.array(pic)
            processed_image = preprocess_input(numpy_image)
            X_list.append(processed_image)
            y_list.append(class_)
        
    X = np.array(X_list)
    y = np.array(y_list)
    
    return X, y, classes

In [ ]:
X,y,classes= load_image(base_path)

In [ ]:
X.shape

In [ ]:
y

In [ ]:
my_dict = {"wallet":0, "phone":1}

In [ ]:
y = np.vectorize(my_dict.get)(y)
y